Testing out the functionality within the QuantLib library to see if it will be of use for this amortisation project.

In [1]:
import QuantLib as ql
import pandas as pd

In [40]:
ql.Settings.instance().evaluationDate = ql.Date(15, 2, 2008)
effectiveDate = ql.Date(15, 2, 2008)
maturityDate = ql.Date(15, 11, 2016)
tenor = ql.Period(ql.Semiannual)
calendar = ql.UnitedKingdom()
businessConvention = ql.Unadjusted
dateGeneration = ql.DateGeneration.Backward
monthEnd = False
schedule = ql.Schedule(effectiveDate, maturityDate, tenor, calendar, businessConvention, businessConvention, dateGeneration, monthEnd)
print(list(schedule))

[Date(15,2,2008), Date(15,5,2008), Date(15,11,2008), Date(15,5,2009), Date(15,11,2009), Date(15,5,2010), Date(15,11,2010), Date(15,5,2011), Date(15,11,2011), Date(15,5,2012), Date(15,11,2012), Date(15,5,2013), Date(15,11,2013), Date(15,5,2014), Date(15,11,2014), Date(15,5,2015), Date(15,11,2015), Date(15,5,2016), Date(15,11,2016)]


In [4]:
# not sure if the above's worked properly, as the settlement date (effectiveDate) is showing as a coupon date, but we'll see.
# build the coupon
dayCount = ql.Thirty360()
couponRate = 0.0575
coupons = [couponRate]

In [32]:
# construct the Fixed Rate Bond (hopefully)
settlementDays = 0
faceValue = 100
fixedRateBond = ql.FixedRateBond(settlementDays, faceValue, schedule, coupons, dayCount)

In [33]:
# add in the clean market price on the effective date
cleanPrice = 95.04287
# and the compunding method
compounding = ql.Compounded
# and change the frequency back to being a number instead of a period
cpn_freq = ql.Semiannual

In [34]:
print(tenor, ql.Semiannual, cpn_freq)
print(fixedRateBond.settlementDate())

6M 2 2
February 15th, 2008


In [35]:
bondyield = fixedRateBond.bondYield(cleanPrice, dayCount, compounding, cpn_freq) * 100
print(bondyield)

6.502942566871642


In [48]:
print(6.5 - bondyield)

-0.0029425668716420716


This is close(-ish) to the actual yield of 6.5%, but as I don't have (and can't seem to find a way to get) the issue date for this bond, I'll have to try this again for one that I *do* have the issue date for.

In [2]:
ql.Settings.instance().evaluationDate = ql.Date(19, 12, 2013)
issue_date = ql.Date(27, 4, 2012)
maturity_date = ql.Date(4, 7, 2044)
period = ql.Period(ql.Annual)
freq = ql.Annual
country_calendar = ql.Germany()
bus_convention = ql.Unadjusted
date_gen = ql.DateGeneration.Backward
month_end = False
div_sched = ql.Schedule(issue_date, maturity_date, period, country_calendar, bus_convention, bus_convention, date_gen, month_end)
print(list(div_sched))

[Date(27,4,2012), Date(4,7,2012), Date(4,7,2013), Date(4,7,2014), Date(4,7,2015), Date(4,7,2016), Date(4,7,2017), Date(4,7,2018), Date(4,7,2019), Date(4,7,2020), Date(4,7,2021), Date(4,7,2022), Date(4,7,2023), Date(4,7,2024), Date(4,7,2025), Date(4,7,2026), Date(4,7,2027), Date(4,7,2028), Date(4,7,2029), Date(4,7,2030), Date(4,7,2031), Date(4,7,2032), Date(4,7,2033), Date(4,7,2034), Date(4,7,2035), Date(4,7,2036), Date(4,7,2037), Date(4,7,2038), Date(4,7,2039), Date(4,7,2040), Date(4,7,2041), Date(4,7,2042), Date(4,7,2043), Date(4,7,2044)]


In [3]:
day_count = ql.ActualActual()
coupon_rate = 0.025
coups = [coupon_rate]

In [4]:
settlement_days = 0
face_value = 100
fix_rate_bd = ql.FixedRateBond(settlement_days, face_value, div_sched, coups, day_count)

In [5]:
clean_price = 96.29
comp = ql.Compounded

In [6]:
bond_yield = fix_rate_bd.bondYield(clean_price, day_count, comp, freq) * 100
print(bond_yield)

2.6789230203628547


Again, this is *agonisingly* close, but not exactly equal to the known value of 2.679016%
Some of the additional features, like business convention, date generation and compounding need to be looked at to see if there's anything that needs adjusting with them.

In [7]:
print(2.679016 - bond_yield)

9.2979637145163e-05


In [8]:
bus_conv2 = ql.Following
div_sched2 = ql.Schedule(issue_date, maturity_date, period, country_calendar, bus_conv2, bus_conv2, date_gen, month_end)
print(list(div_sched2))

[Date(27,4,2012), Date(4,7,2012), Date(4,7,2013), Date(4,7,2014), Date(6,7,2015), Date(4,7,2016), Date(4,7,2017), Date(4,7,2018), Date(4,7,2019), Date(6,7,2020), Date(5,7,2021), Date(4,7,2022), Date(4,7,2023), Date(4,7,2024), Date(4,7,2025), Date(6,7,2026), Date(5,7,2027), Date(4,7,2028), Date(4,7,2029), Date(4,7,2030), Date(4,7,2031), Date(5,7,2032), Date(4,7,2033), Date(4,7,2034), Date(4,7,2035), Date(4,7,2036), Date(6,7,2037), Date(5,7,2038), Date(4,7,2039), Date(4,7,2040), Date(4,7,2041), Date(4,7,2042), Date(6,7,2043), Date(4,7,2044)]


In [9]:
fix_rate_bd2 = ql.FixedRateBond(settlement_days, face_value, div_sched2, coups, day_count)
bond_yield2 = fix_rate_bd2.bondYield(clean_price, day_count, comp, freq) * 100
print(bond_yield2)

2.6790084409713755


In [10]:
print(2.679016 - bond_yield2)

7.559028624370967e-06


closer still...

In [11]:
sett_days = 2
fix_rate_bd3 = ql.FixedRateBond(sett_days, face_value, div_sched, coups, day_count)
bond_yield3 = fix_rate_bd3.bondYield(clean_price, day_count, comp, freq) * 100
print(bond_yield3)

2.6789633131027233


In [12]:
print(2.679016 - bond_yield3)

5.2686897276554845e-05


maybe a little further away...

In [13]:
fix_rate_bd4 = ql.FixedRateBond(sett_days, face_value, div_sched2, coups, day_count)
bond_yield4 = fix_rate_bd4.bondYield(clean_price, day_count, comp, freq) * 100
print(bond_yield4)

2.6790487337112436


In [14]:
print(2.679016 - bond_yield4)

-3.2733711243793095e-05


...and now out on the other side (higher than expected)

In [15]:
sett_days2 = 3
fix_rate_bd5 = ql.FixedRateBond(sett_days2, face_value, div_sched, coups, day_count)
bond_yield5 = fix_rate_bd5.bondYield(clean_price, day_count, comp, freq) * 100
print(bond_yield5)

2.679005217552186


In [16]:
print(2.679016 - bond_yield5)

1.078244781371751e-05


This feels like the closest I've got so far... Ah, not quite - version 2 was closer

In [17]:
fix_rate_bd6 = ql.FixedRateBond(sett_days2, face_value, div_sched2, coups, day_count)
bond_yield6 = fix_rate_bd6.bondYield(clean_price, day_count, comp, freq) * 100
print(bond_yield6)

2.679090638160706


In [18]:
print(2.679016 - bond_yield6)

-7.463816070618634e-05


and that's just getting worse, not better

In [19]:
print(fix_rate_bd.settlementDate())
print(fix_rate_bd2.settlementDate())
print(fix_rate_bd3.settlementDate())
print(fix_rate_bd4.settlementDate())
print(fix_rate_bd5.settlementDate())
print(fix_rate_bd6.settlementDate())

December 19th, 2013
December 19th, 2013
December 23rd, 2013
December 23rd, 2013
December 27th, 2013
December 27th, 2013


So, only 1 and 2 have the correct settlement date showing (both reflect 0 settlement days). 2 is the closer of them, as it has a business convention of using the following day after a holiday.

In [20]:
comp7 = ql.Simple
bond_yield7 = fix_rate_bd2.bondYield(clean_price, day_count, comp7, freq) * 100
print(bond_yield7)

2.6785926198959356


Much further away...

In [21]:
country_calendar8 = ql.NullCalendar()
div_sched8 = ql.Schedule(issue_date, maturity_date, period, country_calendar8, bus_conv2, bus_conv2, date_gen, month_end)
print(list(div_sched8))

[Date(27,4,2012), Date(4,7,2012), Date(4,7,2013), Date(4,7,2014), Date(4,7,2015), Date(4,7,2016), Date(4,7,2017), Date(4,7,2018), Date(4,7,2019), Date(4,7,2020), Date(4,7,2021), Date(4,7,2022), Date(4,7,2023), Date(4,7,2024), Date(4,7,2025), Date(4,7,2026), Date(4,7,2027), Date(4,7,2028), Date(4,7,2029), Date(4,7,2030), Date(4,7,2031), Date(4,7,2032), Date(4,7,2033), Date(4,7,2034), Date(4,7,2035), Date(4,7,2036), Date(4,7,2037), Date(4,7,2038), Date(4,7,2039), Date(4,7,2040), Date(4,7,2041), Date(4,7,2042), Date(4,7,2043), Date(4,7,2044)]


In [22]:
fix_rate_bd8 = ql.FixedRateBond(settlement_days, face_value, div_sched8, coups, day_count)
bond_yield8 = fix_rate_bd8.bondYield(clean_price, day_count, comp, freq) * 100
print(bond_yield8)

2.6790098341457047


In [23]:
print(2.679016 - bond_yield8)

6.165854295137052e-06


The *tiniest* bit closer...

In [29]:
day_count9 = ql.ActualActual(ql.ActualActual.ISMA)
fix_rate_bd9 = ql.FixedRateBond(settlement_days, face_value, div_sched8, coups, day_count9)
bond_yield9 = fix_rate_bd9.bondYield(clean_price, day_count9, comp, freq) * 100
print(bond_yield9)

2.679016499519349


In [30]:
print(2.679016 - bond_yield9)

-4.99519348995392e-07


That is bang on to 6 decimal places. Need to test this for a different date to see if this calculation is more in line with Excel or the old system.

In [31]:
ql.Settings.instance().evaluationDate = ql.Date(3, 3, 2015)
issue_date_2 = ql.Date(27, 4, 2012)
maturity_date_2 = ql.Date(4, 7, 2044)
period_2 = ql.Period(ql.Annual)
freq_2 = ql.Annual
country_calendar_2 = ql.Germany()
bus_convention_2 = ql.Following
date_gen_2 = ql.DateGeneration.Backward
month_end_2 = False
div_sched_2 = ql.Schedule(issue_date_2, maturity_date_2, period_2, country_calendar_2, bus_convention_2, bus_convention_2, date_gen_2, month_end_2)
print(list(div_sched_2))

[Date(27,4,2012), Date(4,7,2012), Date(4,7,2013), Date(4,7,2014), Date(6,7,2015), Date(4,7,2016), Date(4,7,2017), Date(4,7,2018), Date(4,7,2019), Date(6,7,2020), Date(5,7,2021), Date(4,7,2022), Date(4,7,2023), Date(4,7,2024), Date(4,7,2025), Date(6,7,2026), Date(5,7,2027), Date(4,7,2028), Date(4,7,2029), Date(4,7,2030), Date(4,7,2031), Date(5,7,2032), Date(4,7,2033), Date(4,7,2034), Date(4,7,2035), Date(4,7,2036), Date(6,7,2037), Date(5,7,2038), Date(4,7,2039), Date(4,7,2040), Date(4,7,2041), Date(4,7,2042), Date(6,7,2043), Date(4,7,2044)]


In [38]:
day_count_2 = ql.ActualActual(ql.ActualActual.ISMA)
coupon_rate_2 = 0.025
coups_2 = [coupon_rate_2]
settlement_days_2 = 0
face_value_2 = 100
fix_rate_bd_2 = ql.FixedRateBond(settlement_days_2, face_value_2, div_sched_2, coups_2, day_count_2)
clean_price_2 = 139.775 # had been left as 96.29...
comp_2 = ql.Compounded
bond_yield_2 = fix_rate_bd_2.bondYield(clean_price_2, day_count_2, comp_2, freq_2) * 100
print(bond_yield_2)

0.9419778282642368


...works better when you use the correct price...

In [34]:
print(fix_rate_bd_2.settlementDate())

March 3rd, 2015


In [35]:
div_sched_2a = ql.Schedule(ql.Date(3, 3, 2015), maturity_date_2, period_2, country_calendar_2, bus_convention_2, bus_convention_2, date_gen_2, month_end_2)
print(list(div_sched_2a))

[Date(3,3,2015), Date(6,7,2015), Date(4,7,2016), Date(4,7,2017), Date(4,7,2018), Date(4,7,2019), Date(6,7,2020), Date(5,7,2021), Date(4,7,2022), Date(4,7,2023), Date(4,7,2024), Date(4,7,2025), Date(6,7,2026), Date(5,7,2027), Date(4,7,2028), Date(4,7,2029), Date(4,7,2030), Date(4,7,2031), Date(5,7,2032), Date(4,7,2033), Date(4,7,2034), Date(4,7,2035), Date(4,7,2036), Date(6,7,2037), Date(5,7,2038), Date(4,7,2039), Date(4,7,2040), Date(4,7,2041), Date(4,7,2042), Date(6,7,2043), Date(4,7,2044)]


In [37]:
fix_rate_bd_2a = ql.FixedRateBond(settlement_days_2, face_value_2, div_sched_2a, coups_2, day_count_2)
bond_yield_2a = fix_rate_bd_2a.bondYield(clean_price_2, day_count_2, comp_2, freq_2) * 100
print(bond_yield_2a)

0.9422523024082187


further away, so not an issue with the div schedule.

Try changing the country calendar back to NullCalendar()

In [39]:
country_calendar_2b = ql.NullCalendar()
div_sched_2b = ql.Schedule(issue_date_2, maturity_date_2, period_2, country_calendar_2b, bus_convention_2, bus_convention_2, date_gen_2, month_end_2)
fix_rate_bd_2b = ql.FixedRateBond(settlement_days_2, face_value_2, div_sched_2b, coups_2, day_count_2)
bond_yield_2b = fix_rate_bd_2b.bondYield(clean_price_2, day_count_2, comp_2, freq_2) * 100
print(bond_yield_2b)

0.9418416388034822


Agrees to Excel calculation to 6 decimal places (0.941842)

In [42]:
calc_price_2b = fix_rate_bd_2b.cleanPrice(bond_yield_2b / 100, day_count_2, comp_2, freq_2)
print(calc_price_2b)

139.7750135058054


The Price formula is refreshingly simple, once you have the bond set up in the first place.

So, set up the bond once then change the evaluation date for each date that you need to perform calculations for. Maybe define a function and call it via a lambda function to populate a pandas dataframe.

The way forward looks to be: import the data into two dataframes and add all of the extra fields needed by the calculations; iterate through the static dataframe and extract the corresponding data from the trades dataframe to then perform any calculations that may not be easy to achieve in the dataframe format; write the data back into the trades dataframe, potentially using the merge 'SQL-like' function to ensure it is written back in the correct place.

Whether this will be a scalable approach for larger datasets remains to be seen, but Dask is an alternative to pandas if it comes to it.

Let's start with just getting the base functionality sorted out, i.e. performing the amortisation recalculations, then we can move on to scaling issues, etc.

In [160]:
import pandas as pd
import numpy as np
import QuantLib as ql
from numba import jit

In [29]:
# import both sheets / tabs at once, using the ExcelFile wrapper
with pd.ExcelFile(".\Test Data\Test_data_xlsx.xlsx") as xlsx:
    s_df = pd.read_excel(xlsx, "Static")
    t_df = pd.read_excel(xlsx, "Trades")

In [30]:
# add a new column to both dataframes (Fund + Security Code + SEDOL [padded to 7 characters with leading zeroes]) and make it the index
s_df["ID"] = s_df["Fund"] + s_df["Security Code"] + s_df["SEDOL"].astype(str).str.zfill(7)
t_df["ID"] = t_df["Fund"] + t_df["Security Code"] + t_df["SEDOL"].astype(str).str.zfill(7)

In [56]:
t_df

,Fund,Trade Type,Security Code,SEDOL,Contract Date,Settlement Date,Units,Unit Price (Local),Cost (Local)
ID,,,,,,,,,
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2013-12-16,2013-12-19,70000,96.2900,67403.00
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2014-02-03,2014-02-06,30000,100.3030,28887.00
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2014-11-10,2014-11-12,70000,118.0530,82637.10
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,121153.10
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,140382.33
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-02-27,2015-03-03,10000,139.7750,13977.50
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-07-23,2015-07-27,10000,125.8780,12587.80
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-12-04,2015-12-08,20000,124.2940,25684.25
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2016-02-25,2016-02-29,110000,142.5720,141263.38


In [31]:
# set the 'ID' column as the index on both dataframes
s_df.set_index("ID", inplace=True)
t_df.set_index("ID", inplace=True)

In [34]:
s_df

,Security Code,SEDOL,Fund,Issue Date,Maturity Date,PAR Value,Coupon Rate,Frequency,Accrual Method,Days in Month,Days in Year,Bond Type,Calendar Used
ID,,,,,,,,,,,,,
TESTTESTSEC9999999,TESTSEC,9999999,TEST,2012-04-27,2044-07-04,100,0.025,1,1,Actual,Actual,Bullet,NaN


In [38]:
s_df.loc["TESTTESTSEC9999999", "Issue Date"]

Timestamp('2012-04-27 00:00:00')

In [47]:
issue = s_df.loc["TESTTESTSEC9999999", "Issue Date"].date()
iss = ql.Date(issue)
print(issue)
print(iss)

TypeError: Wrong number or type of arguments for overloaded function 'new_Date'.
  Possible C/C++ prototypes are:
    Date::Date()
    Date::Date(Day,Month,Year)
    Date::Date(Day,Month,Year,Hour,Minute,Second,Millisecond,Microsecond)
    Date::Date(Day,Month,Year,Hour,Minute,Second,Millisecond)
    Date::Date(Day,Month,Year,Hour,Minute,Second)
    Date::Date(BigInteger)
    Date::Date(std::string const &,std::string)


Ahhhh... the date formats between the normal Python date and the QuantLib date are inverted. QL takes day, month, year whereas Python shows year-month-day hour:min:sec

To convert from one to the other will either need some involved coding, or a little research to see if there's an easy way to achieve this.

In [48]:
print(issue)

2012-04-27


In [50]:
# the datetime date needs to be formatted as a string for the parser to work.
iss = ql.DateParser.parseFormatted(str(issue), "%Y-%m-%d")
print(iss)

April 27th, 2012


That *seems* to be the quick way to do it. Feels like the dates would be best converted and added back into the table as new columns so they can be used by QL.

In [52]:
# or, the following could be used
iss_date = ql.Date(issue.day, issue.month, issue.year)
print(iss_date)

April 27th, 2012


Yep, the second option is the much simpler way to convert the date - although this still needs the initial conversion of the df data into a datetime format. Is there a one-step method?

In [55]:
iss_date_ql = ql.DateParser.parseFormatted(str(s_df.loc["TESTTESTSEC9999999", "Issue Date"]), "%Y-%m-%d")
print(iss_date_ql)

April 27th, 2012


That's the puppy! Should be simple enough to make into a lambda function for block conversion of the df.

In [57]:
# add the additional columns required to perform the calculations and populate with a placeholder of None
s_df["QL Issue Date"] = None
s_df["QL Maturity Date"] = None
s_df["Tenor"] = None
s_df["Day Count"] = None

In [59]:
# now do the same for the trades df - just one column to add for that
t_df["Evaluation Date"] = None

In [63]:
s_df["QL Issue Date"] = s_df["Issue Date"].apply(lambda x: ql.DateParser.parseFormatted(str(x), "%Y-%m-%d"))
s_df["QL Maturity Date"] = s_df["Maturity Date"].apply(lambda x: ql.DateParser.parseFormatted(str(x), "%Y-%m-%d"))
s_df["Tenor"] = s_df["Frequency"].apply(lambda x: ql.Period(x))

In [66]:
excel_to_ql_basis = {0: ql.Thirty360(), 1: ql.ActualActual(ql.ActualActual.ISMA), 2: ql.Actual360(), 3: ql.Actual365Fixed(), 4: ql.Thirty360(ql.Thirty360.European)}

In [67]:
s_df["Day Count"] = s_df["Accrual Method"].apply(lambda x: excel_to_ql_basis[x])

In [68]:
s_df

,Security Code,SEDOL,Fund,Issue Date,Maturity Date,PAR Value,Coupon Rate,Frequency,Accrual Method,Days in Month,Days in Year,Bond Type,Calendar Used,QL Issue Date,QL Maturity Date,Tenor,Day Count
ID,,,,,,,,,,,,,,,,,
TESTTESTSEC9999999,TESTSEC,9999999,TEST,2012-04-27,2044-07-04,100,0.025,1,1,Actual,Actual,Bullet,NaN,"April 27th, 2012","July 4th, 2044",1Y,Actual/Actual (ISMA) day counter


In [75]:
s_df["Calendar Used"] = s_df["Calendar Used"].apply(lambda x: x if pd.notna(x) else ql.NullCalendar())

In [76]:
s_df

,Security Code,SEDOL,Fund,Issue Date,Maturity Date,PAR Value,Coupon Rate,Frequency,Accrual Method,Days in Month,Days in Year,Bond Type,Calendar Used,QL Issue Date,QL Maturity Date,Tenor,Day Count
ID,,,,,,,,,,,,,,,,,
TESTTESTSEC9999999,TESTSEC,9999999,TEST,2012-04-27,2044-07-04,100,0.025,1,1,Actual,Actual,Bullet,Null calendar,"April 27th, 2012","July 4th, 2044",1Y,Actual/Actual (ISMA) day counter


Now that the static data table is set up, I should be able to construct the bond. I just need to find the best way to extract the data from the table. And to set up the default values where applicable.

In [77]:
# set up the static values used in the calculations
bus_conv = ql.Following
date_gen = ql.DateGeneration.Backward
month_end = False
sett_days = 0
comp = ql.Compounded

In [83]:
# extract data from the dataframe into a pandas Series object
bd_data = s_df.loc["TESTTESTSEC9999999", ["QL Issue Date", "QL Maturity Date", "Tenor", "Frequency", "Calendar Used", "Day Count", "Coupon Rate", "PAR Value"]]

In [84]:
print(bd_data)

QL Issue Date                       April 27th, 2012
QL Maturity Date                      July 4th, 2044
Tenor                                             1Y
Frequency                                          1
Calendar Used                          Null calendar
Day Count           Actual/Actual (ISMA) day counter
Coupon Rate                                    0.025
PAR Value                                        100
Name: TESTTESTSEC9999999, dtype: object


In [87]:
pd.Series(bd_data).get("Tenor")

Period("1Y")

In [88]:
pd.Series(bd_data).get("Day Count")

<QuantLib.QuantLib.ActualActual; proxy of <Swig Object of type 'QuantLib::ActualActual *' at 0x00000260A5FA45A0> >

In [150]:
# allocate the data from the Series / dataframe extract to variables to use in the construction of the bond
iss_date = pd.Series(bd_data).get("QL Issue Date")
mat_date = pd.Series(bd_data).get("QL Maturity Date")
tenor = pd.Series(bd_data).get("Tenor")
freq = int(pd.Series(bd_data).get("Frequency"))
cal = pd.Series(bd_data).get("Calendar Used")
day_count = pd.Series(bd_data).get("Day Count")
# note that the coupon has to be created as a list
coupon = [pd.Series(bd_data).get("Coupon Rate")]
# and the face value has to be an integer
face_val = int(pd.Series(bd_data).get("PAR Value"))

In [118]:
# create the dividend schedule
div_sched = ql.Schedule(iss_date, mat_date, tenor, cal, bus_conv, bus_conv, date_gen, month_end)
print(list(div_sched))

[Date(27,4,2012), Date(4,7,2012), Date(4,7,2013), Date(4,7,2014), Date(4,7,2015), Date(4,7,2016), Date(4,7,2017), Date(4,7,2018), Date(4,7,2019), Date(4,7,2020), Date(4,7,2021), Date(4,7,2022), Date(4,7,2023), Date(4,7,2024), Date(4,7,2025), Date(4,7,2026), Date(4,7,2027), Date(4,7,2028), Date(4,7,2029), Date(4,7,2030), Date(4,7,2031), Date(4,7,2032), Date(4,7,2033), Date(4,7,2034), Date(4,7,2035), Date(4,7,2036), Date(4,7,2037), Date(4,7,2038), Date(4,7,2039), Date(4,7,2040), Date(4,7,2041), Date(4,7,2042), Date(4,7,2043), Date(4,7,2044)]


In [130]:
# construct the bond
fix_rate_bd = ql.FixedRateBond(sett_days, face_val, div_sched, coupon, day_count)

Other than working out a way to iterate over the dataframe without needing to input the row index each time, this *appears* to be the static data side of it complete - for now

next, add the extra columns onto the trade dataframe and see where to go from here.

In [131]:
# add the additional columns needed for the yield calculation, plus some for the amortisation calculation, and set to a default of None
# already added the Evaluation Date when adding the columns to the static data dataframe
t_df["Redemption Yield"] = None
t_df["Adjusted Units"] = None
t_df["Current Holding"] = None
t_df["Blended Yield"] = None

In [133]:
t_df

,Fund,Trade Type,Security Code,SEDOL,Contract Date,Settlement Date,Units,Unit Price (Local),Cost (Local),Evaluation Date,Redemption Yield,Adjusted Units,Current Holding,Blended Yield
ID,,,,,,,,,,,,,,
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2013-12-16,2013-12-19,70000,96.2900,67403.00,None,None,None,None,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2014-02-03,2014-02-06,30000,100.3030,28887.00,None,None,None,None,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2014-11-10,2014-11-12,70000,118.0530,82637.10,None,None,None,None,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,121153.10,None,None,None,None,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,140382.33,None,None,None,None,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-02-27,2015-03-03,10000,139.7750,13977.50,None,None,None,None,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-07-23,2015-07-27,10000,125.8780,12587.80,None,None,None,None,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-12-04,2015-12-08,20000,124.2940,25684.25,None,None,None,None,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2016-02-25,2016-02-29,110000,142.5720,141263.38,None,None,None,None,None


In [139]:
# same date reformatting as earlier
t_df["Evaluation Date"] = t_df["Settlement Date"].apply(lambda x: ql.DateParser.parseFormatted(str(x), "%Y-%m-%d"))
# using numPy to implement a conditional based on the Trade Type column
t_df["Adjusted Units"] = np.where(t_df["Trade Type"]=="SALE", -t_df["Units"], t_df["Units"])
# using the pandas cumsum() method to create the running total for the holding
t_df["Current Holding"] = t_df["Adjusted Units"].cumsum()
t_df

,Fund,Trade Type,Security Code,SEDOL,Contract Date,Settlement Date,Units,Unit Price (Local),Cost (Local),Evaluation Date,Redemption Yield,Adjusted Units,Current Holding,Blended Yield
ID,,,,,,,,,,,,,,
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2013-12-16,2013-12-19,70000,96.2900,67403.00,"December 19th, 2013",None,70000,70000,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2014-02-03,2014-02-06,30000,100.3030,28887.00,"February 6th, 2014",None,-30000,40000,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2014-11-10,2014-11-12,70000,118.0530,82637.10,"November 12th, 2014",None,70000,110000,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,121153.10,"January 1st, 2015",None,-110000,0,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,140382.33,"January 1st, 2015",None,110000,110000,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-02-27,2015-03-03,10000,139.7750,13977.50,"March 3rd, 2015",None,10000,120000,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-07-23,2015-07-27,10000,125.8780,12587.80,"July 27th, 2015",None,10000,130000,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-12-04,2015-12-08,20000,124.2940,25684.25,"December 8th, 2015",None,-20000,110000,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2016-02-25,2016-02-29,110000,142.5720,141263.38,"February 29th, 2016",None,-110000,0,None


In [203]:
# this just leaves the yield and blended yield calculations
# taking the yield first...and defining a function for it, as the evaluation date needs to be changed for each calculation
def red_yield(eval_date, clean_price):
    ql.Settings.instance().evaluationDate = eval_date
    ry = fix_rate_bd.bondYield(clean_price, day_count, comp, freq)
    return ry

# then apply the function where the trade type is a purchase
t_df["Redemption Yield"] = np.where(t_df["Trade Type"]=="PURCHASE", t_df.apply(lambda x: red_yield(x["Evaluation Date"], x["Unit Price (Local)"]), axis=1), 0)

In [204]:
t_df

,Fund,Trade Type,Security Code,SEDOL,Contract Date,Settlement Date,Units,Unit Price (Local),Cost (Local),Evaluation Date,Redemption Yield,Adjusted Units,Current Holding,Blended Yield
ID,,,,,,,,,,,,,,
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2013-12-16,2013-12-19,70000,96.2900,67403.00,"December 19th, 2013",0.026790,70000,70000,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2014-02-03,2014-02-06,30000,100.3030,28887.00,"February 6th, 2014",0.000000,-30000,40000,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2014-11-10,2014-11-12,70000,118.0530,82637.10,"November 12th, 2014",0.017175,70000,110000,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,121153.10,"January 1st, 2015",0.000000,-110000,0,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,140382.33,"January 1st, 2015",0.013575,110000,110000,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-02-27,2015-03-03,10000,139.7750,13977.50,"March 3rd, 2015",0.009418,10000,120000,None
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-07-23,2015-07-27,10000,125.8780,12587.80,"July 27th, 2015",0.014054,10000,130000,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-12-04,2015-12-08,20000,124.2940,25684.25,"December 8th, 2015",0.000000,-20000,110000,None
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2016-02-25,2016-02-29,110000,142.5720,141263.38,"February 29th, 2016",0.000000,-110000,0,None


...and now to work out how to calculate the blended yield...

In [205]:
# this is the idea behind the weighted average...
blend_y = ((t_df.iloc[4,10] * t_df.iloc[4,12]) + (t_df.iloc[5,10] * t_df.iloc[5,6])) / t_df.iloc[5, 12]
print(blend_y)

0.013228341869910563


...and it agrees to the Excel version. Now to convert the above into a function that can work dynamically.

In [191]:
t_df["Redemption Yield"].shape

(12,)

In [208]:
#@jit
def blend_yield(trade_type, curr_holding, units, redm_yield):
    b_yield = np.empty(redm_yield.shape)
    b_yield[0] = redm_yield[0] 
    for i in range(1, b_yield.shape[0]):
        if trade_type[i] == "PURCHASE" and curr_holding[i] - units[i] == 0:
            b_yield[i] = redm_yield[i]
            #return b_yield[i]
        elif trade_type[i] == "SALE":
            b_yield[i] = b_yield[i-1]
            #return b_yield[i]
        else:
            b_yield[i] = ((b_yield[i-1] * curr_holding[i-1]) + (redm_yield[i] * units[i])) / curr_holding[i]
            #return b_yield[i]
    return b_yield

t_df["Blended Yield"] = blend_yield(*t_df[["Trade Type", "Current Holding", "Units", "Redemption Yield"]].values.T)

In [209]:
#t_df["Blended Yield"] = None
t_df

,Fund,Trade Type,Security Code,SEDOL,Contract Date,Settlement Date,Units,Unit Price (Local),Cost (Local),Evaluation Date,Redemption Yield,Adjusted Units,Current Holding,Blended Yield
ID,,,,,,,,,,,,,,
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2013-12-16,2013-12-19,70000,96.2900,67403.00,"December 19th, 2013",0.026790,70000,70000,0.026790
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2014-02-03,2014-02-06,30000,100.3030,28887.00,"February 6th, 2014",0.000000,-30000,40000,0.026790
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2014-11-10,2014-11-12,70000,118.0530,82637.10,"November 12th, 2014",0.017175,70000,110000,0.020671
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,121153.10,"January 1st, 2015",0.000000,-110000,0,0.020671
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-01-01,2015-01-01,110000,127.6203,140382.33,"January 1st, 2015",0.013575,110000,110000,0.013575
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-02-27,2015-03-03,10000,139.7750,13977.50,"March 3rd, 2015",0.009418,10000,120000,0.013228
TESTTESTSEC9999999,TEST,PURCHASE,TESTSEC,9999999,2015-07-23,2015-07-27,10000,125.8780,12587.80,"July 27th, 2015",0.014054,10000,130000,0.013292
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2015-12-04,2015-12-08,20000,124.2940,25684.25,"December 8th, 2015",0.000000,-20000,110000,0.013292
TESTTESTSEC9999999,TEST,SALE,TESTSEC,9999999,2016-02-25,2016-02-29,110000,142.5720,141263.38,"February 29th, 2016",0.000000,-110000,0,0.013292


And that's the table complete... Next up is extracting the data for the relevant recalculation period and performing the amortisation recalculation.